In [331]:
import requests
import json
import pprint
import pandas as pd
import csv
import re
from datetime import datetime
import time

# I. Lấy thông tin chung
- Lấy thông tin của tất cả các quán ăn thuộc 1 tỉnh/TP. Rộng hơn là thông tin tất cả quán ăn của nhiều tỉnh thành
- Với mỗi quán ăn, lấy những trường thông tin cơ bản như
    + name, addr, CityId, RestaurantId, RestaurantStatus <font color=green>// thông tin cơ bản</font>
    + AvgRating
    + TotalPictures, TotalSaves, TotalReviews, TotalFavourites, TotalViews, TotalCheckIns
    + CreatedOn <font color=green>//ngày shop dc tạo đầu tiên trên foody (trường này bị sao rồi, toàn mặc định)</font>
    + Latitude & Longitude <font color=green>//toạ độ</font>
    + ResUrlReviews <font color=green>//URL dẫn tới trang chứa bình luận của shop đó</font>


In [332]:
## nháp
page=1
st=9
all_shops_API = requests.get(f"https://www.foody.vn/ho-chi-minh/dia-diem?vt=row&st={st}&dt=1&page={page}", 
                            headers={'X-Requested-With': 'XMLHttpRequest',
                                     'Cookie': 'FOODY.AUTH=A3F33DD0BA0DC443960AC294DA1D898EE203A64E31B64523D3674F29D60EF3C3C900F1F3BF70DAC47FD0ED268F012BD8FD06CCA143E3FCC2F79AA22CFF031D91A06F5D97B96B09529CF8E5A06E58BCF34312AFDA1F37E6B39CC56A6BA525D72DEDDF1EFE136334F777A1AD191BC3B393CA5D7B3008AA54A9565C21793E196AB15D19E8867327553F696DDCA2938C22208B84620BAA2F757C9D4B8E2FF3CE8A28E6EBC891AAE2F227C02CF6834427BD815163F0DAB98F2E47CECEA44EE3CB88F9A94D33123EEA15B660CF6DE09385DF9EAFD8741C8B4936BF4B5C40B003150E622D303BAC626EBF7FD60BCB19647A46FC',
                                    })
data = all_shops_API.text 
dataJson = json.loads(data)

# print(len(dataJson['searchItems']))
dataJson['searchItems']

[{'Address': '30 Lê Lợi, P. Bến Nghé',
  'District': 'Quận 1',
  'City': 'TP. HCM',
  'Phone': None,
  'Promotions': [],
  'Cuisines': [{'Id': 21,
    'Name': 'Món Nhật',
    'NameEn': None,
    'ASCIIName': None,
    'DetailUrl': '/ho-chi-minh/dia-diem-phong-cach-nhat-ban'},
   {'Id': 36,
    'Name': 'Món Á',
    'NameEn': None,
    'ASCIIName': None,
    'DetailUrl': '/ho-chi-minh/dia-diem-phong-cach-chau-a'}],
  'SpecialDesc': None,
  'TotalReview': 1365,
  'TotalView': 238227,
  'TotalFavourite': 1768,
  'TotalCheckins': 589,
  'AvgRating': '7.7',
  'AvgRatingOriginal': 7.726,
  'ReviewUrl': '/ho-chi-minh/morico-modern-japanese-restaurant-cafe-le-loi/binh-luan',
  'AlbumUrl': '/ho-chi-minh/morico-modern-japanese-restaurant-cafe-le-loi/album-anh',
  'Latitude': 10.774454,
  'Longitude': 106.700742,
  'MainCategoryId': 2,
  'PictureCount': 12488,
  'Status': 2,
  'IconUrl': None,
  'FriendAction': {'TotalCount': 0, 'FriendActions': []},
  'HasAlredyAddedToList': False,
  'AdsProvider

In [328]:
def get_single_page(page, st, dist):
    '''extract 1 page'''
    url = f'https://www.foody.vn/ho-chi-minh/dia-diem?vt=row&st={st}&dt={dist}&page={page}'
    all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                               'Cookie': 'FOODY.AUTH=A3F33DD0BA0DC443960AC294DA1D898EE203A64E31B64523D3674F29D60EF3C3C900F1F3BF70DAC47FD0ED268F012BD8FD06CCA143E3FCC2F79AA22CFF031D91A06F5D97B96B09529CF8E5A06E58BCF34312AFDA1F37E6B39CC56A6BA525D72DEDDF1EFE136334F777A1AD191BC3B393CA5D7B3008AA54A9565C21793E196AB15D19E8867327553F696DDCA2938C22208B84620BAA2F757C9D4B8E2FF3CE8A28E6EBC891AAE2F227C02CF6834427BD815163F0DAB98F2E47CECEA44EE3CB88F9A94D33123EEA15B660CF6DE09385DF9EAFD8741C8B4936BF4B5C40B003150E622D303BAC626EBF7FD60BCB19647A46FC'})
    data = all_shops_API.text
    json_data = json.loads(data)
    if not json_data['searchItems']:
        return []
    shop_attribs = []
    
    for shop in json_data['searchItems']: 
        shop_attribs.append({'ReviewUrl' : shop['ReviewUrl'],
                            'Address': shop['Address']}) # bsung 
    
    return shop_attribs # [{},{},..]

def get_single_district(dist=1):
    '''get all shops in a single district'''
    page = 1
    all_shop = []
    empty = False

    for page in range(1,50):
        if empty:
            break
        print(f'page {page}')
        st = 1

        for st in range(1,50):
            print(f'  st = {st}')
            time.sleep(1)
            one_page = get_single_page(page, st, dist) 
            
            if not one_page: # check empty
                empty = True
                break
            all_shop.extend(one_page)
    return all_shop # [{},{},..]                       

def get_single_city(city_id='217'):
    all_dist = '1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,2,18,693,696,699,695,694,698'.split(',')
    all_city_shops = []

    for dist in all_dist:
        all_city_shops.extend(get_single_district(dist))
    return all_city_shops

In [ ]:
shops_in_a_district = get_single_district(dist=1)
print(len(shops_in_a_district))

df = pd.DataFrame.from_dict(shops_in_a_district) 
print(f'lọc trùng:', df.drop_duplicates().shape)
df.drop_duplicates().to_csv(r'allshop.csv', index = False, header=True, encoding='utf-8')

# II. Lấy thông tin cụ thể từng quán ăn
Thông tin lần này chia thành 2 loại:
- Thông tin thực đơn (Get delivery dishes)
    - dish_type_name <font color=green>//Loại đồ ăn: 1 loại đồ ăn gồm nhiều món ăn, VD: bún thì có bún cá, bún chả</font>
    - total_like <font color=green>//tổng lượt like loại đồ ăn đó</font>
    - is_available <font color=green>//loại đồ ăn đó còn món ko</font>
    - dish_name <font color=green>//món ăn thuộc loại đồ ăn đó. VD: bún cá thập cẩm</font>
    - dish_price <font color=green>//giá 1 món ăn thuộc 1 loại đồ ăn </font>
    - options: <font color=green>// có option nào cho món ăn đó. VD: có option chọn thêm topping như rau, cua,... </font>
        [
        option_1: name, price;
        option_2: name, price;
        ....
        ]
    - option_min_select, option_max_select

- Thông tin chi tiết quán ăn (Get detail):
    - phone 
	- district_id
	- category <font color=green>//loại hình quán ăn?</font>
	- cuisines <font color=green>//quán ăn thuộc phân khúc món nào? món Việt, Âu,...?</font>
	- service_fee <font color=green>//fí dịch vụ</font>
	- avg_price <font color=green>// chắc là giá trung bình của quán ăn.</font>
	- min_order_value <font color=green>//fải mua tối thiểu bnhiêu tiền?</font>
	- promotion <font color=green>// trong này nhiều trường nhỏ, tự nghiên cứu thêm</font>
	- min_charge <font color=green>//fí fụ thu nhỏ nhất</font>
	- week_days, operating <font color=green>//thời gian mở cửa</font>
	- min_shipping_fee <font color=green>// có thể là giá ship thấp nhất</font>
	- is_foody_delivery <font color=green>// có thể là shop này có phải là shop bán mang về hay không</font>
	- price_range <font color=green>// khoảng giá min – max của quán.</font>

---

- Get menu of ONE shop: fải loop qua từng request_id trong cell code trên

In [ ]:
# VD: có thể thay request_id = 1031704 để thử
def get_delivery_dishes(restaurant_id):
        delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={restaurant_id}&id_type=1", 
                                        headers={'x-foody-api-version': '1',
                                                'x-foody-app-type': '1004',
                                                'x-foody-client-language': 'vi',
                                                'x-foody-client-type': '1',
                                                'x-foody-client-version': '1',
                                                'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                                }
                                        )
        data = delivery_dish_API.text
        dataJson = json.loads(data)
        print(dataJson)

- get detail of ONE shop

In [ ]:
def get_detail(restaurant_id)
delivery_dish_API = requests.get("https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={restaurant_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)